In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from io import BytesIO
import requests

## Planning Area Data & Spatial Join with the Counting Stations Data

In [2]:
# Let's first load the planning area shapefile
planning_area_gdf = gpd.read_file("lor_planungsraeume_2021/lor_planungsraeume_2021.shp")

# And convert the PLR_ID to integer
planning_area_gdf["PLR_ID"] = planning_area_gdf["PLR_ID"].astype(int)

# And, preview the planning areas
print(planning_area_gdf.head())

     PLR_ID                  PLR_NAME BEZ       STAND   GROESSE_M2  \
0  11501341            Karlshorst Süd  11  01.01.2021  2294021.723   
1   7601340  Tirschenreuther Ring Ost  07  01.01.2021   413969.326   
2   2500831               Wismarplatz  02  01.01.2021   248988.291   
3  12601134        Märkisches Zentrum  12  01.01.2021  1127855.841   
4   7601547        Horstwalder Straße  07  01.01.2021   941433.954   

                                            geometry  
0  POLYGON ((400826.929 5814871.126, 400825.707 5...  
1  POLYGON ((387795.701 5807587.153, 387789.731 5...  
2  POLYGON ((396010.552 5819339.57, 396021.567 58...  
3  POLYGON ((387367.428 5828619.763, 387315.862 5...  
4  POLYGON ((391807.174 5805331.9, 391808.186 580...  


In [3]:
# Let's load and convert bicycle counting stations to geodataframe
bike_df = pd.read_csv("daily_cycling_data_berlin_12032025.csv")

In [4]:
# Let's take out the unique stations with their latitude and longitude
bike_stations = bike_df[['station_name', 'latitude', 'longitude']].drop_duplicates()

# Now, let's convert the bike stations to geodataframe
bike_stations["geometry"] = [Point(xy) for xy in zip(bike_stations["longitude"], bike_stations["latitude"])]
bike_gdf = gpd.GeoDataFrame(bike_stations, geometry="geometry", crs="EPSG:4326")

# Now, let's reproject to match the planning area CRS
bike_gdf = bike_gdf.to_crs(planning_area_gdf.crs)

# And now perform a spatial join to get planning area ID
bike_gdf = gpd.sjoin(bike_gdf, planning_area_gdf[['PLR_ID', 'geometry', 'PLR_NAME']], how="left", predicate="within")

# And, also check how the merge looks now
print(bike_gdf.head())

# And preview the bike stations data
bike_stations.head()

            station_name   latitude  longitude  \
0         Alberichstraße  52.492535  13.558493   
3106     Berliner Straße  52.566870  13.412328   
5907    Breitenbachplatz  52.466988  13.308763   
8708   Frankfurter Allee  52.513687  13.474328   
11478    Invalidenstraße  52.527431  13.372561   

                             geometry  index_right    PLR_ID  \
0      POINT (402132.169 5816798.371)          214  10300734   
3106   POINT (392391.127 5825274.121)          307   3400826   
5907   POINT (385111.922 5814324.812)          103   6400844   
8708   POINT (396468.029 5819268.336)          513   2500729   
11478  POINT (389597.028 5820947.827)          268   1200628   

                 PLR_NAME  
0            Biesdorf Süd  
3106      Tiroler Viertel  
5907               Dahlem  
8708   Pettenkofer Straße  
11478   Lüneburger Straße  


,station_name,latitude,longitude,geometry
0,Alberichstraße,52.492535,13.558493,POINT (13.558493 52.492535)
3106,Berliner Straße,52.566870,13.412328,POINT (13.412328 52.5668695)
5907,Breitenbachplatz,52.466988,13.308763,POINT (13.3087625 52.466988)
8708,Frankfurter Allee,52.513687,13.474328,POINT (13.474328 52.513687)
11478,Invalidenstraße,52.527431,13.372561,POINT (13.3725605 52.5274315)


## Download Land Use Data for Berlin

In [5]:
# Let's define the wfs url
WFS_URL = "https://gdi.berlin.de/services/wfs/ua_flaechennutzung_2015"

# And, also the parameters
PARAMS = {
    "SERVICE": "WFS",
    "VERSION": "2.0.0",
    "REQUEST": "GetFeature",
    "TYPENAMES": "ua_flaechennutzung_2015:c_ua_realnutz_2015",
    "SRSNAME": "EPSG:25833",
    "OUTPUTFORMAT": "application/json"
}

response = requests.get(WFS_URL, params=PARAMS)

if response.status_code == 200:
    land_use_gdf = gpd.read_file(BytesIO(response.content))
    print("Land use data successfully downloaded")
else:
    raise ValueError(f"Failed to download land use data. HTTP {response.status_code}")

Land use data successfully downloaded


## Merge Land Use with the Bike GDF dataset

In [6]:
# Now, let's reproject to meters (EPSG:25833)
land_use_gdf = land_use_gdf.to_crs(epsg=25833)
planning_area_gdf = planning_area_gdf.to_crs(epsg=25833)

# And, compute area of each land use polygon
land_use_gdf["land_use_area_m2"] = land_use_gdf.geometry.area

# And, perform a spatial join: assign each land use polygon to a planning area
land_use_joined = gpd.sjoin(
    land_use_gdf, 
    planning_area_gdf[["PLR_ID", "geometry"]], 
    how="inner", 
    predicate="intersects"
)

# And, check the unique land use types
print(land_use_joined["nutzung"].unique())

['Wohnnutzung' 'Verkehrsfläche (ohne Straßen)'
 'Gemeinbedarfs- und Sondernutzung' 'Ver- und Entsorgung'
 'Gewerbe- und Industrienutzung, großflächiger Einzelhandel'
 'Park / Grünfläche' 'Mischnutzung' 'Kerngebietsnutzung' 'Friedhof'
 'Brachfläche, vegetationsfrei' 'Gewässer' 'Sportnutzung'
 'Stadtplatz / Promenade' 'Brachfläche, wiesenartiger Vegetationsbestand'
 'Brachfläche, Mischbestand aus Wiesen, Gebüschen und Bäumen' 'Baustelle'
 'Kleingartenanlage' 'Baumschule / Gartenbau'
 'Wochenendhaus- und kleingartenähnliche Nutzung' 'Wald' 'Grünland'
 'Ackerland']


In [7]:
# Now, let's map 'nutzung' to standard categories
nutzung_map = {
    "Wohnnutzung": "residential",
    "Verkehrsfläche (ohne Straßen)": "traffic",
    "Gewässer": "waterways",
    "Gewerbe- und Industrienutzung, großflächiger Einzelhandel": "industry",
    "Kleingartenanlage": "private_gardening",
    "Wochenendhaus- und kleingartenähnliche Nutzung": "private_gardening",
    "Park / Grünfläche": "parks",
    "Friedhof": "cemeteries",
    "Wald": "forests",
    "Grünland": "farming",
    "Ackerland": "farming",
    "Baumschule / Gartenbau": "horticulture",
}

# And, apply mapping and filter out unmapped
land_use_joined["land_use_category"] = land_use_joined["nutzung"].map(nutzung_map)
land_use_filtered = land_use_joined.dropna(subset=["land_use_category"])

In [8]:
# Now, let's aggregate area per planning area and category
land_use_summary = (
    land_use_filtered
    .groupby(["PLR_ID", "land_use_category"])["land_use_area_m2"]
    .sum()
    .reset_index()
)

# And, pivot to wide format (columns = land use categories)
land_use_pivot = (
    land_use_summary
    .pivot(index="PLR_ID", columns="land_use_category", values="land_use_area_m2")
    .fillna(0)
)

# ANd, join with total planning area sizes
planning_areas_size = planning_area_gdf.set_index("PLR_ID")[["GROESSE_M2"]]
land_use_pivot = land_use_pivot.join(planning_areas_size)

In [9]:
# Let's convert land use area to percentages
for col in land_use_pivot.columns:
    if col != "GROESSE_M2":
        land_use_pivot[col] = (land_use_pivot[col] / land_use_pivot["GROESSE_M2"]) * 100


In [10]:
# And, merge land use data with bike stations
bike_gdf = bike_gdf.merge(land_use_pivot, on="PLR_ID", how="left")

In [11]:
# ANd, save the resulting dataset
bike_gdf.to_file("bike_stations_with_land_use.geojson", driver="GeoJSON")
bike_gdf.to_csv("bike_stations_with_land_use.csv", index=False)

In [12]:
# Let's also preview merged dataset
print(bike_gdf.head())

        station_name   latitude  longitude                        geometry  \
0     Alberichstraße  52.492535  13.558493  POINT (402132.169 5816798.371)   
1    Berliner Straße  52.566870  13.412328  POINT (392391.127 5825274.121)   
2   Breitenbachplatz  52.466988  13.308763  POINT (385111.922 5814324.812)   
3  Frankfurter Allee  52.513687  13.474328  POINT (396468.029 5819268.336)   
4    Invalidenstraße  52.527431  13.372561  POINT (389597.028 5820947.827)   

   index_right    PLR_ID            PLR_NAME  cemeteries   farming   forests  \
0          214  10300734        Biesdorf Süd    1.127642  1.032912  6.015082   
1          307   3400826     Tiroler Viertel    0.000000  0.000000  0.000000   
2          103   6400844              Dahlem    0.394201  3.340836  6.907066   
3          513   2500729  Pettenkofer Straße    0.000000  0.000000  0.000000   
4          268   1200628   Lüneburger Straße    0.000000  0.000000  0.000000   

   horticulture   industry      parks  private_gar

In [13]:
# And, check for missing land use data
missing_lu = bike_gdf[land_use_pivot.columns.drop("GROESSE_M2")].isnull().sum()
print("Missing land use data per category:\n", missing_lu)

Missing land use data per category:
 cemeteries           0
farming              0
forests              0
horticulture         0
industry             0
parks                0
private_gardening    0
residential          0
traffic              0
waterways            0
dtype: int64


In [14]:
# And, also check the quick stats: average land use per station
print("\nAverage land use percentage across all stations:")
print(bike_gdf[land_use_pivot.columns.drop("GROESSE_M2")].mean().round(2))


Average land use percentage across all stations:
cemeteries            1.99
farming               0.22
forests               1.55
horticulture          0.33
industry             14.23
parks                14.27
private_gardening     2.17
residential          37.47
traffic               7.78
waterways             8.60
dtype: float64


## Summary of this file:

To extract and analyze land use data in relation to Berlin’s bicycle counting stations, a multi-step geospatial data processing workflow was followed using Python. The process began by loading a shapefile representing Berlin’s planning areas, which are neighborhood-level spatial units used for urban planning. Each planning area includes attributes like a unique ID, name, and total area size. Separately, data for bicycle counting stations—each with a unique name, latitude, and longitude—was loaded and converted into a GeoDataFrame. These stations were spatially joined with the planning areas to determine which station falls within which planning zone. This ensured that each bike station could be associated with the characteristics of the surrounding neighborhood.

Next, land use data was retrieved via a Web Feature Service (WFS) request from Berlin’s open geodata portal. The dataset includes detailed polygons labeled with land use types such as residential, industrial, green space, traffic infrastructure, and more. The geometries were reprojected into a metric coordinate system (EPSG:25833) to accurately calculate the area of each land use polygon. A spatial join was then performed to associate each land use polygon with the planning area it intersects. The land use categories, originally in German, were mapped to ten standardized categories including residential, parks, industry, traffic, waterways, cemeteries, forests, farming, horticulture, and private gardening. For each planning area, the total area of land use polygons falling into these categories was aggregated. These absolute values were then normalized by dividing by the total size of the respective planning area to produce percentage values—indicating, for example, that 25% of a given planning area is covered by parks.

The final output is a dataset in which each bicycle station is enriched with the surrounding land use composition of its planning area. This includes percentage values for each of the ten standardized land use categories, in addition to original station attributes like name, coordinates, and planning area ID.